# LSEG Data Libraries API

**Sentiment Scoring for News**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial covers **Natural Language Processing (NLP)** based on news from the Eikon Data API:

* Reading News Headlines
* Extracting and Storing Raw Texts
* Sentiment Scoring Examples
* Sentiment Scoring Over Time
* Combining Sentiment with Index Levels

## Imports and Versions

The following imports several **packages** as used in the following.

In [24]:
import numpy as np  # NumPy numerical computing
import pandas as pd  # pandas Data Analysis
import nltk, bs4  # NLP toolkit & BeautyfulSoup
import lseg.data as ld  # the LSEG Data libraries
import cufflinks as cf  # interactive plotting
import pickle

from bs4 import BeautifulSoup  # HTML parsing
from nltk.sentiment.vader import SentimentIntensityAnalyzer  # sentiment analysis
import configparser as cp

If necessary, download required files for `nltk`.

In [3]:
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to C:\Users\Marios
[nltk_data]     Skevofylakas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Marios
[nltk_data]     Skevofylakas\AppData\Roaming\nltk_data...


True

The following **Python and package versions** are used.

In [4]:
import sys
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [5]:
np.__version__

'1.22.1'

In [6]:
pd.__version__

'1.3.5'

In [8]:
ld.__version__

'2.0.0'

In [9]:
cf.__version__

'0.17.3'

In [10]:
nltk.__version__

'3.7'

In [11]:
bs4.__version__

'4.12.3'

## Connecting to LSEG Data Libraries API

In [82]:
ld.open_session()

<lseg.data.session.Definition object at 0x13440ba0310 {name='workspace'}>

## Reading News Headlines

The function `ld.news.get_headlines()` allows you to search for and retrieve **news headlines**, including `storyId` values needed to retrieve the full news text.

A `query` string might contain `RICs` and other words to be searched for.

In [14]:
news = ld.news.get_headlines('R:TSLA.O PRODUCTION',
                         start='2024-02-15',
                         end='2024-03-16',
                         count=20
                        )

In [38]:
news.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 20 entries, 2024-03-15 16:06:40.909000 to 2024-03-12 06:44:56.490000
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   headline    20 non-null     string
 1   storyId     20 non-null     string
 2   sourceCode  20 non-null     string
dtypes: string(3)
memory usage: 640.0 bytes


In [16]:
news.head()

headline  \
versionCreated                                                               
2024-03-15 16:06:40.909  Tesla Model 2 set for production next year as ...   
2024-03-15 13:05:39.309  Tesla's Cybertruck Production Surges, Defying ...   
2024-03-15 06:12:41.280  Tesla Model 2 set for production later this ye...   
2024-03-15 01:20:12.658  Musk Talks Tesla Semi & Lower Priced Car Produ...   
2024-03-14 07:25:07.275  Musk visits German Tesla plant as production r...   

                                                           storyId sourceCode  
versionCreated                                                                 
2024-03-15 16:06:40.909     urn:link:webnews:20240315:nNRAs369cm:0  NS:AUTOCA  
2024-03-15 13:05:39.309     urn:link:webnews:20240315:nNRAs3402s:0  NS:WEBPRO  
2024-03-15 06:12:41.280     urn:link:webnews:20240315:nNRAs2zcm8:0  NS:AUTOCA  
2024-03-15 01:20:12.658     urn:link:webnews:20240315:nNRAs2wzm8:0  NS:CLECON  
2024-03-14 07:25:07.275  urn:newsml:newsroom:20240314:nNRAs2lt8f:0  NS:WASTIM

## Collecting Raw Texts

The analyses that follow are based on **all news stories** as identified above. To this end, the raw texts are collected in a `list` object (or loaded if such a file exists).

In [ ]:
import pickle

In [40]:
%%time
baseurl = "/data/news/v1/stories/"
stories = []
try:
    news = pickle.load(open('lseg_news.pkl', 'rb'))
except:
    for storyId in news['storyId']:
        request_definition = ld.delivery.endpoint_request.Definition(
            url = baseurl + storyId,
            method = ld.delivery.endpoint_request.RequestMethod.GET
        )
        response = request_definition.get_data()

        rawr = response.data.raw
        if 'newsItem' in rawr.keys():
            storyText = rawr['newsItem']['contentSet']['inlineData']['$']
            stories.append(storyText)
        else:
            stories.append('')
    news['story'] = stories    
    pickle.dump(news, open('lseg_news.pkl', 'wb'))

Wall time: 8.31 s


## Sentiment Scoring Examples

First, a `SentimentIntensityAnalyzer` object is instantiated.

In [18]:
sid = SentimentIntensityAnalyzer()

The following example illustrates a **negative sentiment** score.

In [19]:
scores = sid.polarity_scores(
    '''This is absolute rubbish. I really didn't like. It was so bad.''')
scores

{'neg': 0.433, 'neu': 0.567, 'pos': 0.0, 'compound': -0.7833}

This one yields a **positive sentiment** score.

In [20]:
scores = sid.polarity_scores(
    '''I really liked it. It was amazing. Real fun.''')
scores

{'neg': 0.0, 'neu': 0.329, 'pos': 0.671, 'compound': 0.8805}

In the same way, a sentiment scoring can be implemented for a news text.

In [43]:
text = news.story[5]
text[200:350]

'rlin with his son after the factory resumed production following a week-long pause caused by an arson attack.\n\nWhat Happened: "We are back," Tesla wro'

In [44]:
scores = sid.polarity_scores(text)
scores

{'neg': 0.05, 'neu': 0.91, 'pos': 0.04, 'compound': -0.6249}

## Sentiment Over Time

The following code scores all retrieved news texts for sentiments and collects the results in a `DataFrame` object.

In [66]:
sentiment = pd.DataFrame()

In [67]:
sentiment_list = []
for storyId in news['storyId']:
    row = news[news['storyId'] == storyId]
    scores = sid.polarity_scores(row['story'][0])
    sentiment = sentiment.append(pd.DataFrame(scores, index=row.index))

In [71]:
sentiment.sort_index(inplace=True)

In [72]:
sentiment.head()

neg    neu    pos  compound
versionCreated                                        
2024-03-12 06:44:56.490  0.000  0.000  0.000    0.0000
2024-03-12 07:16:22.100  0.000  0.000  0.000    0.0000
2024-03-13 00:00:00.000  0.117  0.872  0.011   -0.9590
2024-03-13 00:00:00.000  0.077  0.923  0.000   -0.7003
2024-03-13 07:26:08.823  0.040  0.922  0.038   -0.0516

Some statistics about the **sentiment scores** for all news texts.

In [73]:
sentiment.describe()

neg       neu        pos   compound
count  20.000000  20.00000  20.000000  20.000000
mean    0.048100   0.53360   0.018350  -0.409750
std     0.050365   0.44851   0.020559   0.462919
min     0.000000   0.00000   0.000000  -0.992100
25%     0.000000   0.00000   0.000000  -0.950000
50%     0.040000   0.85100   0.005500  -0.051600
75%     0.096250   0.88150   0.038500   0.000000
max     0.132000   1.00000   0.049000   0.000000

The **frequency distribution** for the compounded sentiment scores.

In [74]:
sentiment['compound'].iplot(kind='histogram', bins=15)

The compounded sentiment scores **over time** for raw values.

In [75]:
sentiment['compound'].iplot(mode='markers')

The cumulative compounded sentiment scores **over time** for raw values.

In [76]:
sentiment['compound'].cumsum().iplot(mode='markers')

The cumulative compounded sentiment scores **over time** with transformed values (to **`+1` and `-1`**).

In [77]:
sentiment['transform'] = sentiment['compound'].apply(lambda x: 1 if x > 0 else -1)

In [78]:
sentiment['transform'].value_counts()

-1    20
Name: transform, dtype: int64

In [79]:
sentiment['transform'].cumsum().iplot(mode='markers')

## Combining Sentiment with Index Levels

First, retrieve **historical closing index levels** for the S&P 500.

In [83]:
data = ld.get_history('.SPX',  # the RICs
                      fields='TRDPRC_1',  # the required fields
                      start='2024-02-15',  # start date
                      end='2024-03-16') 

Second, **resample the cumulative compounded sentiment scores** to end-of-day and forward fill the empty rows.

In [84]:
daily = sentiment['compound'].cumsum().resample('D', label='right').last().ffill()

Third, **join** the cumulative compounded sentiment scores with the historical index levels and **plot** the two time series (with different scaling).

In [85]:
data.join(daily).iplot(secondary_y='compound', width=2.5)

In [86]:
ld.close_session()

## Conclusions

This tutorial covers the following **natural language processing (NLP)** tasks based on the Eikon Data API and respective Python packages:

* Reading News Headlines
* Extracting and Storing Raw Texts
* Sentiment Scoring Examples
* Sentiment Scoring Over Time
* Combining Sentiment with Index Levels

## LSEG Data API Developer Resources

* [API Pages](https://developers.lseg.com/en/api-catalog) 
* [Q&A Forums](https://community.developers.refinitiv.com/index.html) 

Data Item Browser Application: Type `DIB` into Workspace Search Bar.

* [Article on Chains](https://developers.lseg.com/en/article-catalog/article/simple-chain-objects-ema-part-1)